<a href="https://colab.research.google.com/github/Shahaf221b/ATP-PartC/blob/master/BuildingIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## General imports

In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

#rom ‏‏inverted_index_colab_anchor import *
from inverted_index_colab_anchor import *
from inverted_index_colab_body import *
from inverted_index_colab_title import *


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Installing, importing, and initializing PySpark


In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 54.3 MB/s 
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 100 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64

In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

## Copy some wiki data

In [5]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [6]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
| [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


# Processing wikipedia

In [7]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)


In [8]:
ids = [23862, 23329, 53672527, 21356332, 4920126, 5250192, 819149, 46448252, 83036, 88595, 18942, 696712, 2032271, 1984246, 5204237, 645111, 18384111, 3673376, 25061839, 271890, 226402, 2380213, 1179348, 15586616, 50278739, 19701, 3596573, 4225907, 19160, 1235986, 6908561, 3594951, 18805500, 5087621, 25049240, 2432299, 381782, 9603954, 390263, 317752, 38007831, 2564605, 13370873, 2403126, 17402165, 23678545, 7837468, 23954341, 11505904, 196698, 34292335, 52042, 2247376, 15858, 11322015, 13062829, 38833779, 7800160, 24193668, 440018, 54351136, 28887886, 19620, 23045823, 43003632, 746577, 1211612, 8305253, 14985517, 30796675, 51800, 964717, 6146589, 13024, 11583987, 57294217, 27471338, 5479462,
35458904, 2720954, 54415758, 233488, 12487489, 48364486, 487132, 416589, 27051151, 66414222, 18985040, 4141563, 376707, 67092078, 61624594, 59591015, 49681, 55052729, 31663650, 35757264, 63995662, 3461736, 56499662, 43003632, 26685, 10147369, 24932989, 66982525, 5323, 46207323, 8495, 31915311, 51552534, 64057049, 45443335, 37849704, 42149032, 63198823, 38833779, 26880450, 39171985, 5213, 44133735, 28486111, 48972018, 23943140, 17740009, 49954680, 28326718, 63016369, 56023027, 58731, 55207134, 62798611, 65732625, 58255666, 68181697, 47892671, 1309220, 65499427, 58255600, 47638279, 46626475, 46374359, 53343108, 173332, 7671, 18745015, 50565707, 57143357, 45112545, 63148404, 50533783, 51112472, 49281083, 51505979, 1181008, 56731447, 59116880, 51578025, 21462612, 1765779, 39675445, 42253,
21035, 36984150, 2702930, 25045060, 24224679, 2555865, 36579642, 310429, 22352499, 11495285, 22294424, 234876, 40748148, 69893, 61962436, 62871079, 843361, 7362700, 16982268, 15712244, 5690287, 7362738, 600236, 12410589, 26584776, 3332410, 20038918, 739855, 1015919, 14201682, 24361010, 53035710, 22901459, 57672434, 4206029, 738384, 36579839, 188521, 15325435, 3602651, 40428462, 322197, 19592340, 3868233, 2385806, 2933438, 23174077, 14001660, 2425344, 288328, 21381229, 26585811, 12652799, 322210, 51078678, 621531, 685130, 11193835, 21197980, 21078348, 3108484, 692988, 31556991, 18741438, 3053003, 50977642, 55115883, 17208913, 64269900, 54077917, 36666029, 50083054, 28245491, 5692662, 18353587, 1994895, 21364162, 20208066, 38574433, 910244, 6154091, 67754025, 2132969, 61386909, 18600765, 579516,
7089, 6693851, 6672660, 23159504, 49770662, 167891, 2399742, 100710, 76723, 5290678, 54229, 3881415, 3720007, 32652613, 1471058, 5239060, 1421165, 1730071, 1277918, 7419133, 17720623, 1765026, 19079925, 1979873, 497794, 57947, 15007729, 85655, 4250574, 2386481, 228541, 55225594, 318873, 22631033, 27767967, 7061714, 8175846, 3881265, 3046256, 606737, 845137, 16161419, 3098266, 54573, 11323402, 936243, 39280615, 13687674, 47155656, 7151675, 43627595, 26879832, 43098662, 2333593, 349448, 2052211, 4432842, 56412300, 1411016, 2152015, 3502051, 33372192, 61422444, 2385217, 1217747, 24315397, 7082459, 856246, 6050655, 27162455, 52140446, 37243595, 36961531, 245067, 1148978, 1770825, 976322, 10300434, 7249348, 14945749, 62851606, 637004, 16224368, 18509922,
25659792, 50404581, 29178, 3080697, 90267, 2568868, 3450096, 49647732, 462173, 43911472, 20229, 40478583, 56643734, 21257512, 2387344, 59405867, 1330188, 12638661, 501757, 446865, 4468939, 25215235, 456363, 95411, 30916, 53487581, 30876926, 301932, 47764842, 426522, 579040, 54155622, 60535326, 23852947, 4275382, 67279077, 16591942, 334546, 602599, 3735620, 10296674, 858120, 30876121, 443480, 1038309, 50653758, 2258995, 34121672, 5382150, 884056, 3141956, 349722, 6745, 3511512, 35211682, 611752, 66963891, 43977806, 36742560, 2899729, 5413930, 61742595, 6972293, 14926, 42674415, 193957, 1950442, 3396753, 2269888, 40055348, 63609800, 1343426, 26078050, 44808, 42444204, 35034191, 9303405, 4627535, 59407816, 28732, 1187122, 6984468, 3328852, 56313776, 43853813, 3533082, 1032674, 39797382, 8892877,
50404581, 301932, 30876121, 12727781, 37018026, 382599, 56232, 1032674, 202437, 9759063, 2047222, 11309920, 15015154, 46482, 406363, 39813131, 9785087, 47840259, 230716, 2240648, 49380722, 62153, 64083, 5355, 1330224, 17345999, 36969, 50577743, 31919750, 1197035, 2175, 15003673, 54145741, 42801, 458008, 47150650, 67922, 39390739, 901091, 28771786, 6984468, 416752, 382619, 198153, 49065540, 496821, 3112548, 15434651, 237489, 47862672, 915309, 708662, 21699434, 550448, 22893145, 17055183, 32863238, 13824676, 1300923, 746225, 33065713, 47770304, 32593, 11002, 20156275, 22735258, 31156754, 32587, 2077960, 344611, 5652480, 31425310, 991758, 40956516, 594987, 1093416, 26473291, 877461, 340356, 13679, 1558639, 11832915, 926863, 14953848, 42155809, 54923, 276975, 66554, 5775715, 3133549, 1104286, 26951370, 3240723, 6523448, 3902658,
44674524, 1092923, 12431, 224584, 43419164, 9874319, 4338696, 3190097, 9651444, 33321947, 26334893, 1497849, 47489893, 32639051, 2030607, 60903, 19133401, 42411494, 47799755, 4028754, 42960975, 24386465, 5339378, 1494648, 14181749, 58582001, 10062721, 33039125, 286747, 48736239, 33367993, 50575063, 55633178, 29403992, 34113322, 3660182, 25173473, 62438513, 60904, 3235536, 40867519, 5376827, 44424763, 58708106, 64302888, 187946, 29156200, 46426771, 35847782, 5376796, 25295524, 33768164, 46551547, 773423, 5376868, 49931371, 59539691, 22411575, 35326347, 46493906, 11451897, 10619416, 36891093, 40116717, 23533163, 879962, 502593, 3371574, 43194901, 41815118, 35673556, 6575642, 43997189, 5913182, 2126501, 52840911, 1566175, 42951365, 42694174, 22992426, 466299, 736238, 7301470, 1431181, 48653985, 51328172, 12003767,
15271, 494530, 442684, 19988623, 731640, 24997830, 10179411, 16635934, 33407925, 11486091, 50716473, 35804330, 18550455, 21106742, 4694434, 26591446, 296950, 24963841, 346470, 509628, 261193, 28688670, 10218640, 1897206, 39000674, 17785794, 38156944, 9586885, 743971, 1185840, 7872152, 10328235, 36794719, 509624, 5818361, 25935906, 22254915, 4881262, 39585214, 30882491, 57312392, 3781784, 25959000, 14109784, 10818378, 25957127, 9511414, 6422823, 20289869, 15101979, 48317971, 14343887, 762092, 4840292, 25130414, 7602386, 37131566, 6118940, 56598843, 11184711, 1315248, 12101316, 22809006, 29979321, 149354, 32817039, 25271852, 20948989, 36749242, 26143506, 19542049, 360030, 20632884, 24963451, 30874683, 11647367, 383162, 13200719, 1981660, 53123104, 10715937, 24019253, 25050663, 27511028, 1514191,
22093, 16795291, 65166616, 65785063, 835946, 890793, 3921, 450389, 20455, 987153, 240940, 246185, 9000355, 5608488, 3280233, 3505049, 5958023, 72852, 8806795, 1811390, 2423824, 516570, 15392541, 72893, 412214, 278018, 12106552, 42846434, 12754503, 9807715, 4108839, 33328593, 64063961, 7215125, 1811320, 1111137, 5035602, 60483582, 9397801, 255645, 16899, 43376, 72855, 65785040, 72866, 6215230, 4987149, 72878, 16160954, 243389, 64639133, 38958735, 72858, 27196905, 38153033, 1385825, 9733533, 49926096, 4875689, 4750398, 28754077, 43569250, 22092, 72889, 59798759, 49778089, 346029, 8588996, 1956255, 52454088, 25390847, 31667631, 878666, 48695845, 72857, 459304, 27837030, 17107550, 72861, 54859596, 9195892, 6560301, 72875, 72883, 240989, 3196517, 24612090,
34258, 59580357, 315141, 47035547, 626718, 32016617, 59516988, 61583621, 38424, 196789, 734272, 60851303, 988753, 92222, 419015, 60992198, 621809, 83742, 3127300, 23834216, 1381751, 744158, 18911094, 1817874, 59052395, 43562594, 31793047, 6154795, 17452177, 62342792, 13936750, 60805249, 60823333, 59899027, 46719542, 7702313, 60310807, 44270128, 68489731, 60500106, 1652601, 359983, 60300514, 45482242, 34756533, 58520592, 43611227, 47586513, 1775374, 6933106, 13412771, 62549131, 49278248, 62557135, 4523354, 60468426, 666420, 1017009, 35795560, 34843274, 60199413, 16488467, 29828807, 585681, 41757168, 66018807, 61285582, 1226448, 28848113, 11487904, 4242777, 1661867, 10671559, 44035836, 6931929, 60705564, 61690747, 7343803, 9042644, 36991237, 3965223, 63090179, 632990, 1041815, 53087254, 418999, 1632919,
952429, 53916, 4189740, 5132315, 5818788, 35270055, 246674, 28271176, 18952739, 13323241, 1257003, 39562449, 11286, 1440567, 294295, 718110, 5806056, 3587235, 515758, 70129, 11984, 25228985, 1572831, 28125449, 1101651, 8220315, 149463, 63883178, 43937, 432174, 55775421, 2413296, 54019, 1878531, 57559, 1822951, 44568, 10104316, 16671094, 59988, 2068329, 12174150, 18952424, 48340, 83024, 99951, 4390795, 4954263, 3012923, 43344741, 61932070, 1480987, 2520434, 15880608, 7852272, 33084611, 27377596, 57866, 22438472, 8150951, 1265113, 19828134, 76046, 37774, 9270823, 1421709, 2051423, 7173874, 59400, 3606300, 8846, 38980932, 427577, 589513, 147484, 3883838, 29565262, 1527015, 533214, 10669865, 32591, 992052, 4975851, 44629591, 162296, 3513027, 11921706, 9041235, 1068768, 5498413, 591668, 42586177, 1185479, 514894, 3419, 14174833, 71265, 7218230,
63631542, 821829, 633458, 63567907, 14208718, 7912947, 159546, 67759831, 7439323, 64192678, 40909056, 5322079, 63164437, 31728348, 12772, 1402997, 1156703, 1840653, 568164, 1814699, 2945076, 2565663, 442333, 33567480, 468313, 19718702, 987724, 903187, 1485962, 5003908, 4201044, 28086000, 28844729, 1265651, 4659608, 64966775, 55960921, 8970797, 4301719, 35892659, 2248622, 6939163, 48561519, 261396, 34298473, 12263290, 44258772, 25219375, 149426, 6558203, 46567337, 46784964, 6458321, 46576311, 18823362, 48315099, 56440599, 57159776, 74910, 67891964, 1702593, 560306, 1015304, 705756, 7479199, 57772096, 58946599, 1210300, 15716827, 34336876, 9040490, 439102, 264104, 11061915, 39774839, 34735506, 42174581, 66859257, 3293969, 16315, 23166476, 42812440, 66935753, 51990351, 36619752, 59863622, 38404, 64027932,
366477, 21000624, 3252816, 29437250, 2399020, 2675881, 360797, 184979, 20775117, 3233153, 31185151, 67528700, 31265743, 31526583, 157649, 60421513, 58847654, 360789, 5089316, 37168, 29772915, 29448480, 63351334, 3136619, 8076709, 41047099, 48658625, 50394170, 2621344, 25496814, 2997690, 41956157, 8817935, 19283335, 3738164, 60383204, 29449433, 20302560, 664951, 580517, 21434255, 8304348, 21611071, 55449550, 27555169, 28153175, 39614564, 314544, 31102405, 1428872, 19952476, 59540110, 40704884, 193714, 8948452, 49891754, 24008586, 32047012, 1104969, 54632, 31323977, 28253219, 33538385, 177671, 3286723, 16368206, 25351191, 5549127, 184959, 52708523, 728321, 31186864, 57685061, 240148, 22067678, 20827796, 4681, 41958461,
150389, 412643, 48475515, 2775579, 4489, 997303, 250247, 311440, 42669424, 1080173, 2295417, 1243208, 1045845, 1890951, 4668823, 1027896, 7364325, 6613094, 8001012, 45079706, 580715, 5866454, 6383976, 18611260, 28643, 2581217, 5686025, 49998617, 63011479, 54736719, 20913864, 52146943, 34900078, 756765, 344269, 254067, 10916, 1764686, 25211041, 28456222, 35334391, 3845894, 34354998, 403653, 46745914, 11415141, 2252455, 283858, 65217527, 3223727, 42549636, 22546, 581077, 7971671, 21496150, 1104090, 21496038, 12082283, 24385806, 575890, 7105280, 54863336, 36164, 1752731, 3611109, 106121, 50364362, 3039817, 38041, 2968175, 70547, 13502823, 36881878, 702635, 18838, 15179951, 286455, 7139331, 218406, 1130867, 21490868, 1025265, 498971, 12915125, 4819795, 49458922, 23446323, 1244458,
599738, 21905962, 39322520, 2608233, 12234103, 6264793, 6497307, 39303262, 54655786, 3378227, 64516, 41300601, 27898480, 672302, 1793236, 1765938, 1179787, 22503790, 43399069, 6263016, 31468446, 23030371, 1540241, 38904159, 11882165, 43707, 54406331, 3275717, 3781845, 57228339, 312903, 16308445, 8277225, 864249, 3115732, 15190087, 2541491, 1923274, 37400147, 47987665, 978408, 36835927, 2589070, 559356, 19696772, 17069844, 13315204, 34829413, 23454546, 2045055, 311935, 37674, 582735, 52077192, 4463369, 11760348,
2036409, 79732, 35052231, 644781, 50991931, 51729995, 56721897, 60583278, 4512778, 53866975, 31728660, 636344, 48286897, 2550824, 66591573, 1360573, 52882803, 1761526, 56758995, 52971602, 2575380, 59519477, 1343949, 53748793, 66029904, 17284321, 62380392, 8250222, 1044117, 37032671, 51478870, 11216001, 2947322, 60085976, 1291991, 9483388, 11231759, 22513329, 5046302, 35927164, 43610835, 2074655, 52076814, 66846904, 58067594, 3997367, 9547083, 16348889, 16184595, 44972706, 37026554, 36753304, 65601792, 66605355, 34964813, 10873990, 13235623, 46302846, 37297021, 3463398, 32739936, 2337323,
55840499, 40475635, 48308444, 65766671, 175537, 7235116, 164376, 42262371, 60883, 53066659, 989858, 302808, 37331423, 18948312, 4552880, 13612929, 55864774, 35729649, 1350109, 52766120, 147143, 55411770, 6920292, 38539, 29831, 23776, 20840, 50276487, 3470969, 59480957, 617668, 33279717, 63704575, 65693607, 57944031, 27306717, 1732642, 32255505, 37061023, 43379979, 19508643, 58150649, 2381862, 21899417, 40438099, 40030145, 6405787, 504230, 925736, 60446377, 37669357, 44146329, 54647064, 66938917, 64704775, 169672, 40867822, 5230872, 66259016, 67736156, 13690215, 342398, 57579192, 394697, 33757091, 55825336, 1068736, 2009711, 47778072, 41451023, 35306910, 31607, 56789255, 18383702, 1492414, 37901793, 2650227, 7152087, 60587209, 818234, 66814247, 59800766, 2387993, 66255918, 63241742, 4765530, 27375211, 21863425, 618856,
57069491, 65967176, 42163310, 878659, 27306717, 41677925, 1074657, 44240443, 17296107, 60952488, 43603241, 22114132, 46208997, 36450985, 41974555, 56289672, 60616450, 59502488, 33038861, 61699239, 61651800, 39368416, 29129051, 55935213, 54537218, 62372638, 60774345, 63090183, 37497391, 51430647, 67229718, 9110929, 61329320, 44254295, 41974496, 200563, 58481694, 48530084, 56289572, 22144990, 612052, 59162931, 55511148, 55511147, 61073786, 59892, 36484005, 36484254, 66423851, 62482816, 1275470, 5676692, 7927053, 60754840, 26999426, 60744481, 56289553, 60463979, 701741, 60283633, 1129847, 36439749, 4451883, 55511155, 22144721, 45359871, 723126, 43655965, 57275457, 12673434, 43867095, 26763420, 39293265, 15003874, 41668588, 61592102, 67063919, 11891433, 64057670, 61513780, 39345917, 67063906, 1221476, 41008758, 60587000, 7729, 2152196, 5027882, 509738, 403585, 26866372, 1339248, 3473503, 4148655,
9232, 31655286, 4465969, 50230910, 35392555, 17664319, 548182, 148869, 34080, 64487, 12232, 38253534, 1278990, 705310, 1849140, 37882040, 305175, 41731654, 5486819, 38345627, 222491, 48534532, 61371, 5065635, 1273047, 881491, 40797848, 6788, 10000, 595149, 43617005, 64437720, 580320, 24001, 5052869, 5028853, 6112, 43825684, 22910214, 13338662, 16990777, 39902695, 7424280, 2040288, 3945278, 167585, 1632912, 13065408, 59917348, 27218900, 1884312, 4539416, 71899, 38342244, 308868, 12629761, 61402898, 19931851, 66273148, 384286, 5763069, 19437928, 8166922, 4649105, 1525546, 26775738, 671331, 2730731,
828085, 32623, 228190, 25311310, 1183972, 240668, 1808135, 10133081, 9917505, 297576, 10591, 5961225, 12897776, 4105482, 27944015, 7368588, 31591921, 431786, 64085, 12209015, 3342275, 21828, 42416319, 7082459, 48212, 529439, 7089, 704826, 228234, 225870, 3651805, 2837891, 976456, 3214240, 23054130, 54587, 8073864, 40783886, 48936348, 634940, 222266, 62149, 25342571, 167891, 241559, 4456418, 7778546, 50280699, 19886, 1188562, 249116, 6690, 35292743, 60862228, 3546873, 348499, 60088597, 113092, 180251, 7438610, 2668736, 55539, 849869, 3111871, 54840207, 64088, 2005216, 507776, 1124192, 304091, 49090939, 39727947, 1063168, 56069398, 84241, 1670565, 3631020, 566114, 1471058, 4979006, 1902624, 1613163, 654732, 937047, 16749463, 2142071, 30224834, 2208584, 34482422, 41580016, 2190209, 23139600, 24056098, 53469]

ids = set(ids)


In [9]:
selectFromData = parquetFile.select("id", "title", "text", "anchor_text").rdd

In [22]:
doc_text_pairs = selectFromData.filter(lambda x: x[0] in ids)
# for x in doc_text_pairs.take(1):
#   # print(x)
#   print(x[3])


In [30]:

anchors = doc_text_pairs.map(lambda x: (x[0],f"{x[3]}"))
for x in anchors.take(2):
  print(x)
# df_ancor = anchors.toDF()
# df_ancor.show()
# print(anchors.take(1))
# word_counts_anchor = anchors.take(1)(lambda x: x[1])
# print(word_counts_anchor.take(1))
# word_counts_anchor_2 = anchors.map(lambda x: x['text'])
# print(word_counts_anchor_2.take(1))

(2175, '[Row(id=159187, text=\'Apple pie\'), Row(id=172479, text=\'Americana\'), Row(id=1190591, text=\'sirloin steak\'), Row(id=52986, text=\'sauteed\'), Row(id=51258, text=\'onion\'), Row(id=10885, text=\'fries\'), Row(id=3734278, text=\'broccoli\'), Row(id=5985739, text=\'carrot\'), Row(id=11737869, text=\'snow pea\'), Row(id=2035329, text=\'garnished\'), Row(id=5395, text=\'chive\'), Row(id=40975500, text=\'fusion cuisine\'), Row(id=32302631, text=\'global cuisine\'), Row(id=59614694, text=\'Mexican-American\'), Row(id=1558, text=\'American Chinese\'), Row(id=11644740, text=\'regional cuisine\'), Row(id=265366, text=\'ethnic\'), Row(id=6186, text=\'Cajun\'), Row(id=170693, text=\'Louisiana Creole\'), Row(id=2793067, text=\'Native American\'), Row(id=22048, text=\'New England\'), Row(id=61655591, text=\'Algonquian\'), Row(id=2281834, text=\'New Mexican\'), Row(id=1356705, text=\'Pennsylvania Dutch\'), Row(id=6949925, text=\'Soul food\'), Row(id=34273510, text=\'Texan\'), Row(id=3085

number of documents that matches the ids

## Word counts

### Term frequency

In [41]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb','row','text'] # TODO: row, text
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  lst = []
  tokens_without_stopwords = [ x for x in tokens if x not in all_stopwords]
  count = Counter(tokens_without_stopwords)
  for word in count.keys():
    if not all(char.isdigit() for char in word):     
      tf = count[word]
      lst.append((word,(id,tf)))

  return lst

 concatenate the tokens from multiple document into one big list


In [42]:
word_counts_body = doc_text_pairs.flatMap(lambda x: word_count(x[2], x[0]))
word_counts_title = doc_text_pairs.flatMap(lambda x: word_count(x[1], x[0]))

# anchors = doc_text_pairs.map(lambda x: x[3])
# print(anchors.take(1))
word_counts_anchor = anchors.flatMap(lambda x: word_count(x[1],x[0]))
print(word_counts_anchor.take(7))

[('apple', (2175, 14)), ('pie', (2175, 19)), ('americana', (2175, 1)), ('sirloin', (2175, 1)), ('steak', (2175, 8)), ('sauteed', (2175, 1)), ('onion', (2175, 3))]


In [43]:
print(word_counts_body.take(1))
print(word_counts_title.take(1))
print(word_counts_anchor.take(1))

[('apple', (2175, 15))]
[('american', (2175, 1))]
[('apple', (2175, 14))]


In [44]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl, key=lambda x: x[0])

In [45]:
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)

In [46]:
print(postings_body.take(1))
print(postings_title.take(1))
print(postings_anchor.take(1))

[('prominent', [(2175, 4), (4489, 1), (4681, 1), (7729, 1), (9232, 1), (11002, 1), (12232, 1), (22546, 3), (24001, 2), (28643, 1), (29831, 1), (34258, 2), (36969, 1), (38041, 1)])]
[('american', [(2175, 1)])]
[('americana', [(2175, 1)])]


### Document frequency and filteration

Next, we will filter out rare words, words that appear in 10 or fewer documents (when working on the entire corpus, we will increase this threshold to a minimum of 50 documents). 

In [47]:
# postings_filtered = postings.filter(lambda x: len(x[1])>10)

Now, we would like to calculate the df for each word.

takes a posting list RDD and returns an RDD where each element is a (token, df) pair. 

In [48]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  rdd1 = postings.flatMap(lambda x: [(x[0],len(x[1]))])

  return rdd1

In [49]:
w2df_body = calculate_df(postings_body)
w2df_title = calculate_df(postings_title)
w2df_anchor = calculate_df(postings_anchor)

w2df_dict_body = w2df_body.collectAsMap() 
w2df_dict_title = w2df_title.collectAsMap()
w2df_dict_anchor = w2df_anchor.collectAsMap()


In [53]:
# p_body = postings_body('american')
# print(p_body)


## Partitioning and writing the index

We would like to partition the tokens and posting lists into buckets so when we get a query, we know exactly in which bucket to look for its posting list. This can also help parallelize retrieval from the index if we have multiple machines answering queries.

 this partitioning and writing process needs to return information about where each posting list is located (file name and offset) for a token.

In [54]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''

def partition_postings_and_write_body(postings):
  return postings.map(lambda post: (token2bucket_id(post[0]), post)) \
.groupByKey() \
.map(lambda t: InvertedIndexBody.write_a_posting_list((t[0], t[1])))


def partition_postings_and_write_title(postings):
  return postings.map(lambda post: (token2bucket_id(post[0]), post)) \
.groupByKey() \
.map(lambda t: InvertedIndexTitle.write_a_posting_list((t[0], t[1])))

def partition_postings_and_write_anchor(postings):
  return postings.map(lambda post: (token2bucket_id(post[0]), post)) \
.groupByKey() \
.map(lambda t: InvertedIndexAnchor.write_a_posting_list((t[0], t[1])))

In [56]:
# posting_locs_list_title = partition_postings_and_write(postings_title).collect()
posting_locs_list_body = partition_postings_and_write_body(postings_body).collect()
posting_locs_list_title = partition_postings_and_write_title(postings_title).collect()
posting_locs_list_anchor = partition_postings_and_write_anchor(postings_title).collect()
print(posting_locs_list_body)
print(posting_locs_list_title)
print(posting_locs_list_anchor)

[defaultdict(<class 'list'>, {'served': [('102_000.bin', 0)], 'rum': [('102_000.bin', 180)], 'three': [('102_000.bin', 204)], 'afraid': [('102_000.bin', 540)], 'mussels': [('102_000.bin', 558)], 'costs': [('102_000.bin', 576)], 'housewives': [('102_000.bin', 678)], 'ww2': [('102_000.bin', 702)], 'hormones': [('102_000.bin', 708)], 'asia': [('102_000.bin', 738)], 'dogma': [('102_000.bin', 858)], 'semolina': [('102_000.bin', 870)], 'rollercoaster': [('102_000.bin', 894)], 'beauty': [('102_000.bin', 900)], 'congo': [('102_000.bin', 942)], 'santa': [('102_000.bin', 948)], 'wood-fired': [('102_000.bin', 1008)], 'erving': [('102_000.bin', 1014)], 'shins': [('102_000.bin', 1038)], 'yahoo': [('102_000.bin', 1044)], 'governor': [('102_000.bin', 1062)], 'follows': [('102_000.bin', 1104)], 'simulating': [('102_000.bin', 1194)], 'naur': [('102_000.bin', 1200)], 'database': [('102_000.bin', 1206)], 'instructors': [('102_000.bin', 1254)], 'anticarcinogens': [('102_000.bin', 1260)], 'unpasteurized': 

 merge the posting locations into a single dict

In [57]:
#body
super_posting_locs_body= defaultdict(list)
for posting_loc in posting_locs_list_body:
  for k, v in posting_loc.items():
    super_posting_locs_body[k].extend(v)
  
#title
super_posting_locs_title = defaultdict(list)
for posting_loc in posting_locs_list_title:
  for k, v in posting_loc.items():
    super_posting_locs_title[k].extend(v)

# #anchor
super_posting_locs_anchor = defaultdict(list)
for posting_loc in posting_locs_list_anchor:
  for k, v in posting_loc.items():
    super_posting_locs_anchor[k].extend(v)

Putting everything together (posting locations, df) and test that the resulting index is correct. 

In [58]:
#body
# Create inverted index instance
inverted_body = InvertedIndexBody()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs_body
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_dict_body
# write the global stats out
inverted_body.write_index('.', 'index') #TODO: check if correct

#title
inverted_title = InvertedIndexTitle()
inverted_title.posting_locs = super_posting_locs_title
inverted_title.df = w2df_dict_title
inverted_title.write_index('.', 'index') #TODO: check if correct

# #anchor
inverted_anchor = InvertedIndexAnchor()
inverted_anchor.posting_locs = super_posting_locs_anchor
inverted_anchor.df = w2df_dict_anchor
inverted_anchor.write_index('.', 'index') #TODO: check if correct

In [59]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list_body(inverted, w):
  with closing(MultiFileReaderBody()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list


def read_posting_list_title(inverted, w):
  with closing(MultiFileReaderTitle()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list


def read_posting_list_anchor(inverted, w):
  with closing(MultiFileReaderAnchor()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [60]:
p_body = read_posting_list_body(inverted_body, 'prominent')
p_title = read_posting_list_title(inverted_title, 'python')
p_anchor = read_posting_list_anchor(inverted_anchor, 'python')

In [61]:
print(p_body)
print(p_title)
print(p_anchor)

[(2175, 4), (4489, 1), (4681, 1), (7729, 1), (9232, 1), (11002, 1), (12232, 1), (22546, 3), (24001, 2), (28643, 1), (29831, 1), (34258, 2), (36969, 1), (38041, 1)]
[(18942, 1), (19160, 1), (19701, 1), (23862, 1)]
[(18942, 1), (19160, 1), (19701, 1), (23862, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
